### Create cluster

###### Replace %%USER%% with your username in kind configuration file

###### This is required so that your local docker config containing your dockerhub credentials can be mounted into the kind nodes

In [1]:
! sed -i "s/%%USER%%/$USER/g" kind-config.yaml

In [3]:
! kind create cluster --config kind-config.yaml --image kindest/node:v1.23.6 --name kcd-vcluster-demo

Creating cluster "kcd-vcluster-demo" ...
 ✓ Ensuring node image (kindest/node:v1.23.6) 🖼
 ✓ Preparing nodes 📦 📦 📦 📦 7l
 ✓ Writing configuration 📜7l
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
 ✓ Joining worker nodes 🚜7l
Set kubectl context to "kind-kcd-vcluster-demo"
You can now use your cluster with:

kubectl cluster-info --context kind-kcd-vcluster-demo

Have a nice day! 👋


### Setup Metallb

In [4]:
! kubectl apply -f https://raw.githubusercontent.com/metallb/metallb/v0.12.1/manifests/namespace.yaml
! kubectl apply -f https://raw.githubusercontent.com/metallb/metallb/v0.12.1/manifests/metallb.yaml

namespace/metallb-system created
podsecuritypolicy.policy/controller created
podsecuritypolicy.policy/speaker created
serviceaccount/controller created
serviceaccount/speaker created
clusterrole.rbac.authorization.k8s.io/metallb-system:controller created
clusterrole.rbac.authorization.k8s.io/metallb-system:speaker created
role.rbac.authorization.k8s.io/config-watcher created
role.rbac.authorization.k8s.io/pod-lister created
role.rbac.authorization.k8s.io/controller created
clusterrolebinding.rbac.authorization.k8s.io/metallb-system:controller created
clusterrolebinding.rbac.authorization.k8s.io/metallb-system:speaker created
rolebinding.rbac.authorization.k8s.io/config-watcher created
rolebinding.rbac.authorization.k8s.io/pod-lister created
rolebinding.rbac.authorization.k8s.io/controller created
daemonset.apps/speaker created
deployment.apps/controller created


In [20]:
! kubectl get pods -n metallb-system

NAME                         READY   STATUS    RESTARTS   AGE
controller-57fd9c5bb-q5mng   1/1     Running   0          117s
speaker-2l6xh                1/1     Running   0          117s
speaker-4v5xl                1/1     Running   0          117s
speaker-g8t7d                1/1     Running   0          117s
speaker-plfz5                1/1     Running   0          117s


##### Setup ip addr to be used by metallb

In [7]:
%%bash 
docker network inspect -f '{{.IPAM.Config}}' kind

[{172.18.0.0/16  172.18.0.1 map[]} {fc00:f853:ccd:e793::/64  fc00:f853:ccd:e793::1 map[]}]


Set ip ranges from 172.18.0.0/16 and create configmap 

Default metallb configmap manifest uses 172.19.0.0/16 so we replace the ip addr as per our local setup

In [5]:
! wget -O metallb-configmap.yaml https://kind.sigs.k8s.io/examples/loadbalancer/metallb-configmap.yaml 
! sed -i 's/172.19.255.200-172.19.255.250/172.18.255.200-172.18.255.250/g' metallb-configmap.yaml
! kubectl apply -f metallb-configmap.yaml

--2022-05-27 17:58:22--  https://kind.sigs.k8s.io/examples/loadbalancer/metallb-configmap.yaml
Resolving kind.sigs.k8s.io (kind.sigs.k8s.io)... 104.248.158.121, 54.255.56.197, 2400:6180:0:d1::637:e001, ...
Connecting to kind.sigs.k8s.io (kind.sigs.k8s.io)|104.248.158.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 218 [application/x-yaml]
Saving to: ‘metallb-configmap.yaml’

metallb-configmap.y 100%[===================>]     218  --.-KB/s    in 0s      

2022-05-27 17:58:22 (50.7 MB/s) - ‘metallb-configmap.yaml’ saved [218/218]

configmap/config unchanged


#### Test LB

In [21]:
! kubectl apply -f https://kind.sigs.k8s.io/examples/loadbalancer/usage.yaml

pod/foo-app created
pod/bar-app created
service/foo-service created


In [26]:
! kubectl get pods,svc 

NAME          READY   STATUS    RESTARTS   AGE
pod/bar-app   1/1     Running   0          23s
pod/foo-app   1/1     Running   0          23s

NAME                  TYPE           CLUSTER-IP     EXTERNAL-IP      PORT(S)          AGE
service/foo-service   LoadBalancer   10.96.13.137   172.18.255.200   5678:30862/TCP   23s
service/kubernetes    ClusterIP      10.96.0.1      <none>           443/TCP          3m29s


In [27]:
%%bash 
LB_IP=$(kubectl get svc/foo-service -o=jsonpath='{.status.loadBalancer.ingress[0].ip}')
echo $LB_IP
for _ in {1..10}; do
  curl -s -S ${LB_IP}:5678
done

172.18.255.200
foo
foo
bar
foo
foo
bar
foo
foo
foo
foo


#### Cleanup cluster

In [2]:
! kind delete cluster --name kcd-vcluster-demo

Deleting cluster "kcd-vcluster-demo" ...


In [10]:
! rm metallb-configmap.yaml